In [14]:
import pandas as pd
import numpy as np

In [15]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)


In [ ]:
from config import (
    N_CUSTOMERS,
    NUM_WEEKS,
    K,
    DECAY_RATE,
    RECOVERY_RATE,
    COOLDOWN_WINDOW,
    COOLDOWN_PENALTY,
    REGRET_THRESHOLD,
    SEEDS
)

In [16]:
from simulation.simulate_week_for_export import initialize_world,advance_one_week

In [ ]:
def run_cooldown():
    df = initialize_world()
    df['last_reviewed_week'] = -1
    df.loc[df['reviewed'],'last_reviewed_week'] = 1

In [ ]:
df = initialize_world()
df['last_reviewed_week'] = -1
df.loc[df['reviewed'],'last_reviewed_week'] = 1

In [ ]:
weekly_regret_cooldown= []
for week in range(2,NUM_WEEKS+1):
    df['adjusted_urgency'] = df['urgency_score']
    recent_mask = ( df['last_reviewed_week'] == week -1 )
    df.loc[recent_mask,'adjusted_urgency'] *= COOLDOWN_PENALTY
    df['reviewed'] = False
    to_review = (
        df.sort_values('adjusted_urgency',ascending=False).head(K).index
    )
    df.loc[to_review,'reviewed'] = True
    df.loc[to_review, 'last_reviewed_week'] = week
    df = advance_one_week(df,DECAY_RATE,RECOVERY_RATE)
    regret_count = ((df['tolerance'] < REGRET_THRESHOLD) & (~df['reviewed'])).sum()
    weekly_regret_cooldown.append(regret_count)

total_regret_cooldown = sum(weekly_regret_cooldown)


In [21]:
df.head(20)

,Customer_id,tolerance,time_gap,worsening_streak,urgency_score,reviewed,prev_time_gap,gap_change,last_reviewed_week,adjusted_urgency
0,0,1.496886,4.371588,0,-0.000000,False,1.863894,-2.507693,9,-0.000000
1,1,0.464288,2.169810,0,-0.000000,False,0.416583,-1.753227,2,1.724501
2,2,0.509535,0.684720,1,2.803417,False,3.488137,2.803417,6,-0.000000
3,3,0.003088,0.010971,1,0.026786,False,0.037757,0.026786,-1,-0.000000
4,4,0.154146,0.087921,2,1.130392,False,0.653117,0.565196,-1,0.994279
5,5,0.085517,0.911749,0,-0.000000,False,0.030962,-0.880786,-1,0.061671
6,6,0.062320,0.181460,1,0.127347,False,0.308807,0.127347,-1,-0.000000
7,7,0.076781,0.737924,0,-0.000000,False,0.171310,-0.566614,-1,0.360094
8,8,0.422781,3.397177,0,-0.000000,False,2.786373,-0.610803,4,-0.000000
9,9,0.358798,0.748048,1,0.972532,False,1.720580,0.972532,-1,-0.000000


In [22]:
df = initialize_world()

In [ ]:
weekly_regret_random = []
for week in range(NUM_WEEKS):
    df['reviewed'] = False
    review_id = np.random.choice(df.index,size=K,replace = False)
    df.loc[review_id,'reviewed'] = True
    df = advance_one_week(df,DECAY_RATE,RECOVERY_RATE)
    regret_count = ((df['tolerance'] < REGRET_THRESHOLD) & (~df['reviewed'])).sum()
    weekly_regret_random.append(regret_count)

total_regret_random = sum(weekly_regret_random)

In [24]:
df = initialize_world()

In [ ]:
weekly_regret_urgency = []
for week in range(NUM_WEEKS):
    df['reviewed'] = False
    review_id = (df.sort_values('urgency_score',ascending=False).head(K).index)
    # print(f"week : {week}| ")
    # print(f"{review_id} | ")
    # print("Week ended")
    df.loc[review_id,'reviewed'] = True
    df = advance_one_week(df,DECAY_RATE,RECOVERY_RATE)
    regret_count = ((df['tolerance'] < REGRET_THRESHOLD) & (~df['reviewed'])).sum()
    weekly_regret_urgency.append(regret_count)

total_regret_urgency = sum(weekly_regret_urgency)

In [26]:
print(f"Random Selection : {total_regret_random}")
print(f"Urgency Selection : {total_regret_urgency}")
print(f"Cooldown Selection : {total_regret_cooldown}")

Random Selection : 385
Urgency Selection : 405
Cooldown Selection : 336
